**Mounting And imports Libararies**

In [1]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import cv2
from random import shuffle
from tqdm import tqdm
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,Dropout, Dense, Input, concatenate,GlobalAveragePooling2D, AveragePooling2D,Flatten
from keras.utils import np_utils
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
import math
from skimage.transform import rotate
from sklearn.model_selection import train_test_split
import torch
from torchvision import transforms
import PIL.Image
from skimage.util import random_noise
from skimage.filters import gaussian

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


**Extract Data**

In [3]:
!mkdir Data
%cd Data
!cp  /content/drive/MyDrive/NN_Dataset.zip /content/Data
!unzip NN_Dataset.zip

/content/Data
Archive:  NN_Dataset.zip
  inflating: Test/0.jpg              
  inflating: Test/1.jpg              
  inflating: Test/10.jpg             
  inflating: Test/100.jpg            
  inflating: Test/101.jpg            
  inflating: Test/102.jpg            
  inflating: Test/103.jpg            
  inflating: Test/104.jpg            
  inflating: Test/105.jpg            
  inflating: Test/106.jpg            
  inflating: Test/107.jpg            
  inflating: Test/108.jpg            
  inflating: Test/109.jpg            
  inflating: Test/11.jpg             
  inflating: Test/110.jpg            
  inflating: Test/111.jpg            
  inflating: Test/112.jpg            
  inflating: Test/113.jpg            
  inflating: Test/114.jpg            
  inflating: Test/115.jpg            
  inflating: Test/116.jpg            
  inflating: Test/117.jpg            
  inflating: Test/118.jpg            
  inflating: Test/119.jpg            
  inflating: Test/12.jpg             
  inflating

**Data PreProcessing**

In [4]:
IMAGE_SIZE = 100
IMAGE_DIM=3

In [5]:
Folder_Train = '/content/Data/Train'
Folder_Test = '/content/Data/Test'
Folder_predictions = '/content/drive/MyDrive/Neural_ProjectFiles/predictions.npy'
Folder_Train_Data = os.listdir(Folder_Train)
Folder_Test_Data = os.listdir(Folder_Test)
Labels = ['Basketball','Football','Rowing','Swimming','Tennis','Yoga']

In [6]:
Data_labels = [np.array([1,0,0,0,0,0]),np.array([0,1,0,0,0,0]),np.array([0,0,1,0,0,0]),np.array([0,0,0,1,0,0]),np.array([0,0,0,0,1,0]),np.array([0,0,0,0,0,1])]
def Create_Labels(Path):
  ImageName = Path.split('_')[0]
  if ImageName =='Basketball':
    return Data_labels[0]
  elif ImageName =='Football':
    return Data_labels[1]
  elif ImageName =='Rowing':
    return Data_labels[2]
  elif ImageName =='Swimming':
    return Data_labels[3]
  elif ImageName =='Tennis': 
    return Data_labels[4]
  elif ImageName =='Yoga':
    return Data_labels[5]


In [7]:
data_validation_path = '/content/drive/MyDrive/Neural_ProjectFiles/validation_data.npy'
cropped_size= 20
def Creat_training_Data(data_training_path):
  training_data = []
  Validation_Data = []
  # shuffle(Folder_Train_Data)
  for _file in Folder_Train_Data:
      image_path = os.path.join(Folder_Train,_file)
      image = cv2.imread(image_path,flags=cv2.IMREAD_COLOR)

      # resize the image and rescale it 
      Image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_AREA)
      # Label Image 
      Class_label = Create_Labels(_file)
      Image=cv2.normalize(Image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
      training_data.append([np.array(Image),Class_label])

  X_train = np.array([i[0] for i in training_data])
  Y_train = np.array([i[1] for i in training_data])

  XData_train,XData_val,YData_train,YData_val= train_test_split(X_train,Y_train,test_size=36,stratify=Y_train,shuffle=True)

  X_train=[]
  Y_train=[]

  
  training_data = []

  Count_class_Data=[]

  Count_class_Data.append(sum(1 for elem in YData_train if (elem == np.array([1,0,0,0,0,0])).all()));
  Count_class_Data.append(sum(1 for elem in YData_train if (elem == np.array([0,1,0,0,0,0])).all()));
  Count_class_Data.append(sum(1 for elem in YData_train if (elem == np.array([0,0,1,0,0,0])).all()));
  Count_class_Data.append(sum(1 for elem in YData_train if (elem == np.array([0,0,0,1,0,0])).all()));
  Count_class_Data.append(sum(1 for elem in YData_train if (elem == np.array([0,0,0,0,1,0])).all()));
  Count_class_Data.append(sum(1 for elem in YData_train if (elem == np.array([0,0,0,0,0,1])).all()));
  
  Max_augmented_label = max(Count_class_Data)
  Count_class_Data=[]
  for index in range(0,len(XData_val)):
      Validation_Data.append([ XData_val[index] , YData_val[index ]])

  Training_Data_List=[]
  for index in range(0,len(XData_train)):
      training_data.append([XData_train[index],YData_train[index]])
      Training_Data_List.append([XData_train[index],YData_train[index]])

  stats = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
  for _data in training_data:
      image=_data[0]
      label_img =_data[1]
      counter = sum(1 for elem in Training_Data_List if ( (elem[1] == label_img).all() ) )
      # Max_Augmented_class = get_MaxAugmented(training_data)
      # if  counter <= Max_Augmented_class:
      # original Image #
      
      if (counter <= (Max_augmented_label + 100) ):

        # Horizontal Flip

        Flip_Horiztonal= np.fliplr(image)
        Training_Data_List.append([Flip_Horiztonal,label_img])

        Flip_up= np.flipud(image)
        Training_Data_List.append([Flip_up,label_img])

     

        Orignal_image = PIL.Image.fromarray(image.astype('uint8'), 'RGB')
        flipped_image = PIL.Image.fromarray(Flip_Horiztonal.astype('uint8'), 'RGB')
        # Augmented = transforms.RandomCrop(IMAGE_SIZE, padding=4, padding_mode='reflect')(Orignal_image)
        # Horizontal = transforms.RandomHorizontalFlip(p=0.9)(Augmented)


        Brighntess_image = np.array(transforms.functional.adjust_gamma(Orignal_image,gamma = 0.5))
        # norm=cv2.normalize(np.array(Brighntess_image), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        Training_Data_List.append([np.array(Brighntess_image),label_img])

        # Make noise in model #
        sigma=0.155
        Noised_image = random_noise(image,var=sigma**2)
        Training_Data_List.append([Noised_image,label_img])


       
        # Make it Darkness #
        darkness_image = np.array( transforms.functional.adjust_gamma(Orignal_image,gamma = 2.5))
        # norm=cv2.normalize(np.array(darkness_image), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        Training_Data_List.append([np.array(darkness_image),label_img])


        rotated_image  = rotate(image, angle=30, mode = 'wrap')
        Training_Data_List.append([rotated_image,label_img])

        rotated_image  = rotate(image, angle=-30, mode = 'wrap')
        Training_Data_List.append([rotated_image,label_img])



        # Blurred The image #
        Blurred_img = gaussian(image,sigma=1,multichannel=True)
        Training_Data_List.append([Blurred_img,label_img])

        # fillped Image #
        rotated_image  = rotate(Flip_Horiztonal, angle=30, mode = 'wrap')
        Training_Data_List.append([rotated_image,label_img])

        rotated_image  = rotate(Flip_Horiztonal, angle=-30, mode = 'wrap')
        Training_Data_List.append([rotated_image,label_img])
        
         # Make noise in model #
        sigma=0.155
        Noised_image = random_noise(Flip_Horiztonal,var=sigma**2)
        Training_Data_List.append([Noised_image,label_img])
        
        darkness_image = np.array( transforms.functional.adjust_gamma(flipped_image,gamma = 2.5))
        # norm=cv2.normalize(np.array(darkness_image), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        Training_Data_List.append([darkness_image,label_img])

        # Brighntess_image = np.array(transforms.functional.adjust_gamma(Orignal_image,gamma = 0.5))
        # # norm=cv2.normalize(np.array(Brighntess_image), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        # Training_Data_List.append([np.array(Brighntess_image),label_img])

        Blurred_img = gaussian(Flip_Horiztonal,sigma=1,multichannel=True)
        Training_Data_List.append([Blurred_img,label_img])


        transformation = transforms.Pad((2, 5, 2, 5))
        translate=transforms.RandomAffine(0, translate=(0.2, 0.2))
        loader_transform2 = transforms.ColorJitter(contrast=2)


        padded= np.array(transformation(Orignal_image))
        padded=cv2.resize(padded,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)

        transleated=np.array(translate(Orignal_image))
        transleated=cv2.resize(padded,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)

        loader_transform2_ = np.array(loader_transform2(Orignal_image))
        loader_transform2_= cv2.resize(padded,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)

        Training_Data_List.append([padded,label_img])
        Training_Data_List.append([transleated,label_img])
        Training_Data_List.append([loader_transform2_,label_img])

        loader_transform = np.array(transforms.Compose([
            transforms.RandomRotation(30),
            transforms.RandomResizedCrop(150),
        ])(flipped_image))
        loader_transform =cv2.resize(loader_transform,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)
        Training_Data_List.append([loader_transform,label_img])

        # rotated_image  = rotate(image, angle=-20, mode = 'wrap')
        # Training_Data_List.append([rotated_image,label_img])
        
        #Convert Image to PIL.Image #
        # Orignal_image = PIL.Image.fromarray(image.astype('uint8'), 'RGB')
       


        #Cropped The image #
        cropped_image = np.array(transforms.functional.center_crop(Orignal_image,output_size=[IMAGE_SIZE-cropped_size]))
        cropped_image=cv2.resize(cropped_image,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)
        Training_Data_List.append([cropped_image,label_img])

        #Cropped Flipped Image #
        # cropped_Filpped_image = np.array(transforms.functional.center_crop(Orignal_image,output_size=[IMAGE_SIZE-cropped_size]))
        # cropped_Filpped_image=cv2.resize(cropped_Filpped_image,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)
        # training_data.append([cropped_Filpped_image,label_img])

        # Rotate 20
        # Image_20_rotation = rotate(image,angle=45)
        # training_data.append([cv2.resize(Image_20_rotation,(IMAGE_SIZE,IMAGE_SIZE)),label_img])

        # # Rotate -20
        # revImg_neg_20_rotation = rotate(_data[0],angle=-20)
        # training_data.append([cv2.resize(revImg_neg_20_rotation,(IMAGE_SIZE,IMAGE_SIZE)),label_img])


      

        # cropped_image = transforms.functional.center_crop(img,output_size=[IMAGE_SIZE-35])
        # final_image = np.array(transforms.functional.resize(cropped_image,size=[IMAGE_SIZE]) )
        # training_data.append([final_image,label_img])

        # darkness_image = np.array( transforms.functional.adjust_gamma(img,gamma = 2.5) )
        # training_data.append([darkness_image,label_img])

  training_data = Training_Data_List
  Training_Data_List= []
  np.save(data_training_path, training_data)
  np.save(data_validation_path, Validation_Data)
  return training_data,Validation_Data

In [8]:
def Create_test_Data(data_test_path):
  test_data = []
  for _file in Folder_Test_Data:
      image_path = os.path.join(Folder_Test,_file)
      image = cv2.imread(image_path,flags=cv2.IMREAD_COLOR)

      # resize the image
      resized_image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE),interpolation=cv2.INTER_AREA)
      
      # Resacle Image 
      resized_image=cv2.normalize(resized_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

      test_data.append([np.array(resized_image),_file])

  np.save(data_test_path, test_data)
  return test_data

In [ ]:
# os.remove('/content/drive/MyDrive/Neural_ProjectFiles/Training_Data.npy')
# os.remove('/content/drive/MyDrive/Neural_ProjectFiles/Test_Data.npy')
os.remove('/content/drive/MyDrive/predictionsFile/predictions.csv')

In [9]:
Training_Data=[]
Validation_Data=[]
labels = []
Test_Data=[]
data_training_path = '/content/drive/MyDrive/Neural_ProjectFiles/Training_Data.npy'
data_test_path = '/content/drive/MyDrive/Neural_ProjectFiles/Test_Data.npy'
# Training Data

# if os.path.exists(data_training_path):
if os.path.exists(data_training_path) and os.path.exists(data_validation_path):
  Training_Data = np.load(data_training_path,allow_pickle=True)
  Validation_Data = np.load(data_validation_path,allow_pickle=True)
else:
  # Training_Data=Creat_training_Data(data_training_path)
  Training_Data,Validation_Data=Creat_training_Data(data_training_path)

print(Labels[0],sum(1 for elem in Training_Data if (elem[1] == np.array([1,0,0,0,0,0])).all()))
print(Labels[1],sum(1 for elem in Training_Data if (elem[1] == np.array([0,1,0,0,0,0])).all()))
print(Labels[2],sum(1 for elem in Training_Data if (elem[1] == np.array([0,0,1,0,0,0])).all()))
print(Labels[3],sum(1 for elem in Training_Data if (elem[1] == np.array([0,0,0,1,0,0])).all()))
print(Labels[4],sum(1 for elem in Training_Data if (elem[1] == np.array([0,0,0,0,1,0])).all()))
print(Labels[5],sum(1 for elem in Training_Data if (elem[1] == np.array([0,0,0,0,0,1])).all()))

shuffle(Training_Data)
shuffle(Validation_Data)
X_train = np.array([i[0] for i in Training_Data]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,IMAGE_DIM)
Y_train = np.array([i[1] for i in Training_Data])
Training_Data = []

X_val = np.array([i[0]for i in Validation_Data]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,IMAGE_DIM)
Y_val = np.array([i[1] for i in Validation_Data])
Validation_Data = []


# Test Data
if os.path.exists(data_test_path):
  Test_Data = np.load(data_test_path,allow_pickle=True)
else:
  Test_Data=Create_test_Data(data_test_path)

X_test = np.array([i[0]  for i in Test_Data]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,IMAGE_DIM)
Y_test = np.array([i[1]  for i in Test_Data])
Test_Data= []

Basketball 519
Football 524
Rowing 525
Swimming 523
Tennis 527
Yoga 523


In [ ]:
print(Labels[0],sum(1 for elem in Y_val if (elem == np.array([1,0,0,0,0,0])).all()))
print(Labels[1],sum(1 for elem in Y_val if (elem == np.array([0,1,0,0,0,0])).all()))
print(Labels[2],sum(1 for elem in Y_val if (elem == np.array([0,0,1,0,0,0])).all()))
print(Labels[3],sum(1 for elem in Y_val if (elem == np.array([0,0,0,1,0,0])).all()))
print(Labels[4],sum(1 for elem in Y_val if (elem == np.array([0,0,0,0,1,0])).all()))
print(Labels[5],sum(1 for elem in Y_val if (elem == np.array([0,0,0,0,0,1])).all()))

Basketball 39
Football 80
Rowing 41
Swimming 48
Tennis 37
Yoga 92


***Training***

In [10]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,filters_5x5_reduce,filters_5x5,filters_pool_proj,name=None):
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    return output

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [11]:
input_layer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE,IMAGE_DIM))

In [12]:
x = Conv2D(128, (3, 3), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(128, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

In [13]:
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96, 
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')

In [14]:
x1 = AveragePooling2D((3, 3), strides=2)(x)
x1 = Conv2D(192, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(6, activation='softmax', name='auxilliary_output_1')(x1)

In [15]:
x =inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')

In [16]:
x2 = AveragePooling2D((3, 3), strides=2)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(6, activation='softmax', name='auxilliary_output_2')(x2)

In [17]:
from keras.backend import flatten
x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
x = Dropout(0.4)(x)
x = Dense(6, activation='softmax', name='output')(x)

In [18]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [ ]:
model.summary()

In [26]:
epochs = 50
initial_lrate = 0.01
def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
opt = tf.keras.optimizers.Adam()
lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [ ]:
num_classes = 6
XData_train,XData_val,YData_train,YData_val = train_test_split(X_train,Y_train,test_size=0.1,random_state=6,stratify=Y_train,shuffle=True)

In [ ]:
X_train = []
Y_train=[]

In [28]:
model.load_weights('/content/drive/MyDrive/ModelFolder/Model.tfl')
# import shutil
# shutil.rmtree('/content/drive/MyDrive/Model_FIles/model_v1.tfl')

In [21]:
Folder_Path = '/content/drive/MyDrive/Model_FIles/model_v1.tfl'
# if os.path.exists(Folder_Path):
#   model.load_weights(Folder_Path)
# else:
XData_train = X_train
YData_train = Y_train
XData_val =X_val 
YData_val = Y_val
Y_train =[]
Y_train =[]
X_train =[]
Y_val =[]

In [ ]:
len(YData_train)

2574

In [ ]:
import torchvision.models as models
googlenet = models.googlenet(pretrained=True)

In [29]:
history = model.fit(XData_train, [YData_train, YData_train, YData_train],validation_data=(XData_val, [YData_val, YData_val, YData_val]), epochs=epochs, batch_size=256,callbacks=[lr_sc])


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/50
13/13 [==============================] - 7s 301ms/step - loss: 0.8587 - output_loss: 0.5357 - auxilliary_output_1_loss: 0.5442 - auxilliary_output_2_loss: 0.5323 - output_accuracy: 0.8844 - auxilliary_output_1_accuracy: 0.8883 - auxilliary_output_2_accuracy: 0.8863 - val_loss: 0.8979 - val_output_loss: 0.6273 - val_auxilliary_output_1_loss: 0.3885 - val_auxilliary_output_2_loss: 0.5135 - val_output_accuracy: 0.8188 - val_auxilliary_output_1_accuracy: 0.8875 - val_auxilliary_output_2_accuracy: 0.8188 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/50
13/13 [==============================] - 3s 245ms/step - loss: 0.5973 - output_loss: 0.3838 - auxilliary_output_1_loss: 0.3506 - auxilliary_output_2_loss: 0.3612 - output_accuracy: 0.8571 - auxilliary_output_1_accuracy: 0.8704 - auxilliary_output_2_accuracy: 0.8628 - val_loss: 0.1051 - val_output_loss: 0.0546 - val_auxilliary_output

**Model Test**

In [31]:
Predictions = model.predict(X_test)

22/22 [==============================] - 2s 43ms/step


In [ ]:
X_test = np.array([i[0]  for i in Test_Data]).reshape(-1,IMAGE_SIZE,IMAGE_SIZE,IMAGE_DIM)
Y_test = np.array([i[1]  for i in Test_Data])

In [32]:
predicted_classes =[]
predicts =[]
index_File=0
for predict in Predictions[0]:
  index = np.argmax(predict)
  predicted_classes.append(Labels[index])
  File_name = Y_test[index_File]
  predicts.append([File_name,index])
  index_File = index_File+1

File_prediction_path='/content/drive/MyDrive/predictionsFile/predictions.csv'

In [ ]:
predicted_classes

In [ ]:
predicts

In [34]:
import pandas as pd

Data = pd.DataFrame(predicts)
Data.columns=['image_name','label']
Data.to_csv(File_prediction_path, index=False)


**Save Model **

In [35]:
model.save('/content/drive/MyDrive/ModelFolder/Model.tfl')